# 01. Data Combination
Combine data from all sources into one **huge** dataframe

In [1]:
# change wd to repo home
import os
if os.getcwd().split('\\')[-1] != 'FM skills prediction':
    os.chdir('..')

In [2]:
import pandas as pd

In [3]:
# load FBRef data in separate dfs
fbref_dfs = {filename : pd.read_csv(f'data/FBRef/{filename}', index_col=0)
             for filename in os.listdir('data/FBRef')}

In [34]:
# combine FBRef dfs by type first
table_types = ['defense', 'gca', 'keepersadv', 'keepers', 'misc', 'passingtypes',
               'passing', 'playingtime', 'possession', 'shooting', 'stats']

fbref_dfs_types = {}

for t in table_types:
    fbref_dfs_types[t] = {}
    for k in fbref_dfs.keys():
        if k.split('_')[1] == t:
            fbref_dfs_types[t][k.split('_')[0]] = fbref_dfs[k]
    fbref_dfs_types[t] = pd.concat(fbref_dfs_types[t], axis=0, ignore_index=True).set_index(['_Player', '_Squad', '_Born'])

In [58]:
# combine FBRef dfs into one
fbref_combined_df = pd.concat(fbref_dfs_types, axis=1).reset_index()
fbref_combined_df.columns = fbref_combined_df.columns.map(lambda x: '_'.join(map(str, x)))

In [77]:
# store a complete FBRef file
fbref_combined_df.to_csv('data/FBRef_combined.csv')

This dataframe still contains multiple duplicates and a crazy number of columns. Duplicates are there becuse some players played for more than one team in a season. Columns are so numerous because I've been lazy and did not bother cleaning them

To deal with player duplicates, we will forst remove players with fewer than 900 minutes (less that 10 '90s')

In [115]:
fbref_combined_filtered_df = fbref_combined_df[fbref_combined_df['misc__90s']<10]
fbref_combined_filtered_df.drop_duplicates(['_Player_'], inplace=True)

C:\Users\alibe\AppData\Local\Temp\ipykernel_46136\2573432723.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fbref_combined_filtered_df.drop_duplicates(['_Player_'], inplace=True)


In [ ]:
# load FM data
fm_fd = pd.read_csv('data/FM 2023.csv')

In [123]:
data_all = fbref_combined_filtered_df.merge(right=fm_fd, left_on='_Player_', right_on='Name', how='inner')

In [124]:
data_all.head()

,_Player_,_Squad_,_Born_,defense__Rk,defense__Nation,defense__Pos,defense__Age,defense__90s,defense_Tackles_Tkl,defense_Tackles_TklW,...,World reputation,Race,RCA,Colour of skin,Date of birth,Number of national team appearances,Goals scored for the national team,Salary,Rental club,UID
0,Sargis Adamyan,Köln,1993,2.0,am ARM,FWMF,29.0,7.2,14.0,7.0,...,4748,North_African__Middle_Eastern,115,7,1993/5/23,34,2,12499.0,NaN,92057768
1,Felix Agu,Werder Bremen,1999,5.0,de GER,DFFW,22.0,0.5,1.0,0.0,...,4599,Mixed_Race,113,10,1999/9/27,0,0,12997.0,NaN,92098106
2,Lucas Alario,Eint Frankfurt,1992,9.0,ar ARG,FWMF,29.0,3.7,3.0,2.0,...,5998,Mediterranean_Hispanic,134,6,1992/10/8,9,3,47944.0,NaN,14046979
3,Faride Alidou,Eint Frankfurt,2001,10.0,de GER,DFMF,21.0,2.0,8.0,5.0,...,3745,African_Caribbean,109,16,2001/7/18,0,0,7614.0,NaN,91206292
4,Lukáš Ambros,Wolfsburg,2004,12.0,cz CZE,MF,18.0,0.2,1.0,0.0,...,3623,Northern_European,102,4,2004/6/5,0,0,4794.0,NaN,2000141427
